In [ ]:
# Cell 1 — Install (run once in notebook if not already installed)
%pip install sentence-transformers langchain faiss-cpu crewai langroid requests
#Note: Installing langroid may require extra steps. If you already have langroid and crewai, skip.



In [ ]:
# Cell 2 — Imports & helpers
import os, time, json, logging, asyncio
from typing import List, Dict, Any

# LLM (Langroid), embeddings & vectordb
from langroid.language_models.openai_gpt import OpenAIGPTConfig, OpenAIGPT
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter
from langchain.docstore.document import Document

# CrewAI (role orchestration)
from crewai import Agent as CrewAgent, Crew, Task as CrewTask

# Basic logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s] %(message)s")
log = logging.getLogger("session2")

# Safe LLM text extractor (Langroid responses vary by version)
# Normalize LLM responses from Langroid/OpenAI/CrewAI into a consistent str
def llm_text(resp) -> str:
    for attr in ("content", "text", "message", "messages"):
        val = getattr(resp, attr, None)
        if val:
            if isinstance(val, list):
                first = val[0]
                if isinstance(first, dict) and "content" in first:
                    return first["content"]
                if hasattr(first, "content"):
                    return getattr(first, "content")
                return str(first)
            if hasattr(val, "content"):
                return getattr(val, "content")
            return str(val)
    return str(resp)


In [ ]:
# Cell 3 — Prepare local RAG knowledge base (example e-commerce docs)
log.info("Preparing a small e-commerce knowledge base...")

docs_texts = [
    "Product: Blue Widget\nSKU: BW-100\nPrice: $19.99\nStock: 10\nDescription: Small blue widget.",
    "Product: Red Widget\nSKU: RW-200\nPrice: $24.99\nStock: 0\nDescription: Red widget, currently out of stock.",
    "Order Policy: Orders are processed within 1 business day. Shipping takes 3-5 business days.",
    "Return Policy: Items may be returned within 30 days if unopened and in original packaging.",
    "Payment Policy: We accept major credit cards. Payment validation may fail for expired cards.",
    "Inventory Update: Inventory updates happen in near real-time after successful order placement.",
    "FAQ: How do I place an order? Use the 'place order' flow with item SKU, quantity, and payment method."
]
docs = [Document(page_content=t) for t in docs_texts]

# Initialize HuggingFaceEmbeddings using model_name
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={"device": "cpu"}
)

vectordb = FAISS.from_documents(docs, embeddings)
log.info("FAISS index built with %d docs", len(docs))


In [ ]:
# Cell 4 — Configure Langroid local LLM
llm_cfg = OpenAIGPTConfig(
    chat_model="local-llm",
    temperature=0.0,
    api_base="http://localhost:1234/v1",  # <- change to your LM Studio / Ollama endpoint
    api_key="not-needed"
)
llm = OpenAIGPT(llm_cfg)
log.info("Langroid LLM wrapper configured for local endpoint")


In [ ]:
# Cell 5 — Tools implemented as functions that use Langroid + vectordb
def retrieval_tool(query: str, k: int = 3) -> List[Dict[str, Any]]:
    log.info("[Tool: retrieval] Query: %s", query)
    hits = vectordb.similarity_search(query, k=k)
    return [{"rank": i+1, "content": h.page_content} for i, h in enumerate(hits)]

def summarizer_tool(text: str) -> str:
    prompt = f"Summarize the following text into 3 concise bullet points:\n\n{text}\n\nSummary:"
    resp = llm.chat(prompt)
    return llm_text(resp)

def file_writer_tool(filename: str, content: str) -> str:
    path = os.path.abspath(filename)
    with open(path, "w", encoding="utf-8") as f:
        f.write(content)
    log.info("[Tool: file_writer] wrote %s (len=%d)", path, len(content))
    return path


In [ ]:
# Cell 6 — LangroidAgent wrapper (context-aware, non-hardcoded)
class LangroidAgent:
    def __init__(self, name: str, llm, context=None):
        self.name = name
        self.llm = llm
        self.tools = {}
        self.context = context or {}

    def register_tool(self, name: str, func):
        self.tools[name] = func
        log.info("[LangroidAgent] Registered tool: %s", name)

    def call_llm(self, prompt: str):
        resp = self.llm.chat(prompt)
        return llm_text(resp)

    def plan(self, user_goal: str) -> List[Dict[str, Any]]:
        """Ask LLM to decompose the goal dynamically into subtasks (JSON)."""
        context_text = json.dumps(self.context, indent=2)

        prompt = f"""
You are an intelligent planner for an e-commerce assistant system.
Decompose the user's goal into an ordered JSON array of subtasks.

Each subtask must have:
  - "role": one of [order, payment, inventory, returns, retrieval, summarizer]
  - "action": what to do
  - "params": a dictionary of parameters for that action

You have access to these tools: {list(self.tools.keys())}.
You can also consider this context (memory, state, history):

{context_text}

User goal: "{user_goal}"

Respond ONLY with valid JSON.
"""

        raw = self.call_llm(prompt)

        try:
            plan = json.loads(raw)
            log.info("[Planner] Parsed %d steps", len(plan))
            # Store in context for future awareness
            self.context.setdefault("recent_plans", []).append(
                {"goal": user_goal, "plan": plan, "time": time.asctime()}
            )
            return plan
        except Exception as e:
            log.warning("[Planner] Failed to parse LLM JSON: %s", e)
            # Fallback: empty plan (no hardcoding)
            return []


In [ ]:
# Cell 7 — register tools on LangroidAgent - planner to remember across sessions - ensures session_history exists
agent_core = LangroidAgent("CoreAgent", llm, context={"session_history": []})
agent_core.register_tool("retrieval", retrieval_tool)
agent_core.register_tool("summarizer", summarizer_tool)
agent_core.register_tool("file_writer", file_writer_tool)


In [ ]:
# Cell 8 — Simple JSON-backed memory using Langroid-style API (context-aware)
MEM_FILE = "langroid_memory.json"
try:
    with open(MEM_FILE, "r", encoding="utf-8") as f:
        memory = json.load(f)
except FileNotFoundError:
    memory = {"conversations": [], "orders": [], "events": []}

def mem_add(key: str, item: dict):
    memory.setdefault(key, []).append(item)
    with open(MEM_FILE, "w", encoding="utf-8") as f:
        json.dump(memory, f, indent=2)
    log.info("[Memory] Added to %s (now %d items)", key, len(memory[key]))


In [ ]:
# Cell 9 — CrewAI agents that call Langroid tools
# Note: CrewAI Agent API may vary. Typical pattern: define an Agent with a run method.
# We'll wrap Langroid tools so each Crew agent simply calls appropriate tool(s).

class CrewAgentWrapper:
    def __init__(self, name: str, role: str, tool_map: Dict[str, Any]):
        self.name = name
        self.role = role
        self.tool_map = tool_map

    async def handle(self, task: Dict[str, Any]) -> Dict[str, Any]:
        log.info("[CrewAgent %s] Handling task: %s", self.name, task)
        role = self.role
        action = task.get("action")
        params = task.get("params", {})

        # routing logic per role & action
        if role == "retrieval":
            q = params.get("query") or params.get("text") or params
            hits = self.tool_map["retrieval"](str(q), k=3)
            return {"status":"ok", "hits": hits}

        if role == "summarizer":
            text = params.get("text") or params.get("content","")
            summary = self.tool_map["summarizer"](text)
            return {"status":"ok", "summary": summary}

        if role == "order":
            sku = params.get("sku")
            qty = int(params.get("quantity", 1))
            # simple inventory check via retrieval
            hits = self.tool_map["retrieval"](f"Product: {sku}")
            stock = None
            for h in hits:
                if "Stock:" in h["content"]:
                    try:
                        stock = int(h["content"].split("Stock:")[1].split()[0])
                    except: stock = None
            if stock is None or stock < qty:
                return {"status":"failed","reason":"insufficient_stock","available":stock}
            order = {"order_id": f"ORD-{int(time.time())}", "sku":sku, "quantity":qty}
            mem_add("orders", order)
            path = self.tool_map["file_writer"](f"order_{order['order_id']}.json", json.dumps(order))
            return {"status":"ok","order": order, "file": path}

        if role == "payment":
            card = params.get("card_number","")
            if not card.isdigit() or len(card) < 12:
                return {"status":"failed","reason":"invalid_card"}
            approved = card.endswith("42") or card.endswith("4242")
            if approved:
                tx = {"tx_id": f"TX-{int(time.time())}", "approved": True}
                mem_add("events", {"type":"payment","tx":tx})
                return {"status":"ok","tx": tx}
            return {"status":"failed","reason":"declined"}

        if role == "inventory":
            sku = params.get("sku"); delta = params.get("quantity", 1)
            ev = {"type":"inventory_update","sku":sku,"delta":-int(delta),"time":time.asctime()}
            mem_add("events", ev)
            return {"status":"ok","event":ev}

        if role == "returns":
            order_id = params.get("order_id")
            found = next((o for o in memory.get("orders",[]) if o.get("order_id")==order_id), None)
            if not found:
                return {"status":"failed","reason":"order_not_found"}
            ev = {"type":"return","order_id":order_id,"time":time.asctime()}
            mem_add("events", ev)
            return {"status":"ok","event": ev}

        return {"status":"failed","reason":"unknown_role"}

# instantiate wrappers for roles
crew_agents = {
    "retrieval": CrewAgentWrapper("Retriever", "retrieval", {"retrieval": retrieval_tool}),
    "summarizer": CrewAgentWrapper("Summarizer", "summarizer", {"summarizer": summarizer_tool}),
    "order": CrewAgentWrapper("OrderAgent", "order", {"retrieval": retrieval_tool, "file_writer": file_writer_tool}),
    "payment": CrewAgentWrapper("PaymentAgent", "payment", {}),
    "inventory": CrewAgentWrapper("InventoryAgent", "inventory", {"file_writer": file_writer_tool}),
    "returns": CrewAgentWrapper("ReturnsAgent", "returns", {})
}


In [ ]:
# Cell 10 — Scheduler that uses CrewAgentWrapper in an async loop (sequential for clarity)
async def run_plan_with_crew(plan: List[Dict[str, Any]]):
    results = []
    log.info("[Scheduler] Running plan with %d steps", len(plan))
    for idx, step in enumerate(plan, start=1):
        role = step.get("role")
        action = step.get("action")
        params = step.get("params", {})
        agent = crew_agents.get(role)
        if not agent:
            res = {"status":"failed","reason":"unknown_role","role":role}
            log.error("Unknown role: %s", role)
        else:
            log.info("Executing step %d: role=%s action=%s params=%s", idx, role, action, params)
            try:
                res = await agent.handle({"action": action, "params": params})
                log.info("Step %d result: %s", idx, res)
            except Exception as e:
                log.exception("Agent handling error")
                res = {"status":"error","error": str(e)}
        results.append({"step": step, "result": res})
    return results


In [ ]:
# Cell 11 — End-to-end handler: plan (Langroid planner) -> Crew scheduler -> summarizer + report
# Async version (Option 2)

async def handle_user_goal_with_crew(user_goal: str):
    log.info("=== New user goal: %s", user_goal)

    # 1) record user goal in planner context
    agent_core.context.setdefault("session_history", []).append(
        {"goal": user_goal, "time": time.asctime()}
    )

    # 2) generate plan
    plan = agent_core.plan(user_goal)
    if not plan:
        log.warning("Planner returned empty plan; aborting")
        return {"error": "empty_plan"}

    # 3) execute plan via Crew (await the async scheduler)
    results = await run_plan_with_crew(plan)

    # 4) summarization
    summary = summarizer_tool(json.dumps(results, indent=2))

    # 5) save report
    report = {
        "goal": user_goal,
        "plan": plan,
        "results": results,
        "summary": summary,
        "timestamp": time.asctime(),
    }
    path = file_writer_tool(
        f"session_report_{int(time.time())}.json", json.dumps(report, indent=2)
    )

    # 6) update context
    agent_core.context.setdefault("last_reports", []).append(
        {"path": path, "goal": user_goal, "time": time.asctime()}
    )

    log.info("Report saved to %s", path)
    return {"plan": plan, "results": results, "summary": summary, "report": path}


In [ ]:
# Cell 12 — Demonstration: single-tool vs multi-tool comparison
# Single-tool baseline: only retrieval + LLM answer (no decomposition, no agents)
def single_tool_answer(user_question: str):
    hits = retrieval_tool(user_question, k=3)
    context = "\n\n".join(h["content"] for h in hits)
    prompt = f"Answer the question based on the context below:\n\n{context}\n\nQuestion: {user_question}"
    resp = llm.chat(prompt)
    return llm_text(resp)

user_goal = "Place an order for 2 units of BW-100 (Blue Widget) and charge card 4242424242424242."
log.info("Running single-tool baseline...")
baseline = single_tool_answer("How to place an order and charge payment for 2 units of BW-100?")

log.info("Running multi-tool crew plan...")
multi_out = await handle_user_goal_with_crew(user_goal)

print("\n--- SINGLE-TOOL BASELINE (excerpt) ---\n", baseline[:800])
print("\n--- MULTI-TOOL PLAN ---\n", json.dumps(multi_out["plan"], indent=2))
print("\n--- MULTI-TOOL RESULTS (summary) ---\n", multi_out["summary"])
print("\nReport saved at:", multi_out["report"])


# To do 1: 

Make the Crew pipeline state-aware, so that:

- If an order fails due to lack of stock → it won’t trigger the payment task.

- If the order succeeds → it updates inventory and proceeds with payment.

We’ll do this by modifying the Crew task orchestration logic (run_plan_with_crew) and adding a small inventory update function.


✅ Step 1. Add a shared inventory store

At the top of your notebook (before defining run_plan_with_crew), define a simple inventory structure:

## Simple shared inventory state (could later be replaced by a DB)
inventory = {
    "BW-100": 1,  # current stock
    "AC-200": 5,
}

✅ Step 2. Define helper functions
def check_inventory(product_id, qty):
    """Return True if inventory is sufficient, else False."""
    available = inventory.get(product_id, 0)
    return available >= qty


def update_inventory(product_id, qty):
    """Deduct stock after successful order."""
    if product_id in inventory:
        inventory[product_id] = max(0, inventory[product_id] - qty)

✅ Step 3. Modify run_plan_with_crew

Locate your existing function (it’s the async part that runs the Crew tasks).
We’ll inject logic between the “order” and “payment” stages:

async def run_plan_with_crew(plan):
    results = []

    for step in plan:
        task = step.get("task", "")
        params = step.get("params", {})

        # Simulated e-commerce logic
        if "order" in task.lower():
            product = params.get("product", "unknown")
            qty = int(params.get("quantity", 1))

            if not check_inventory(product, qty):
                result = {
                    "task": task,
                    "status": "failed",
                    "reason": f"Insufficient stock for {product} (available={inventory.get(product,0)})",
                }
            else:
                update_inventory(product, qty)
                result = {
                    "task": task,
                    "status": "success",
                    "message": f"Order placed for {qty} units of {product}. Inventory updated.",
                }

        elif "payment" in task.lower():
            # ⚠️ Skip payment if last order failed
            last_result = results[-1] if results else {}
            if last_result.get("status") != "success":
                result = {
                    "task": task,
                    "status": "skipped",
                    "reason": "Previous order failed — skipping payment",
                }
            else:
                result = {
                    "task": task,
                    "status": "success",
                    "transaction_id": f"TX-{random.randint(1000000000, 9999999999)}",
                    "message": "Payment processed successfully",
                }

        else:
            # Default dummy execution
            result = {"task": task, "status": "done"}

        results.append(result)

    return results

✅ Step 4. Try it out

Now you can test it by running:

handle_user_goal_with_crew("Place an order for 2 units of BW-100")


Expected summary result:

• Order for product BW-100 failed due to insufficient stock.
• Payment was skipped because the order failed.


Then run again:

handle_user_goal_with_crew("Place an order for 1 unit of BW-100")


Expected:

• Order placed successfully for 1 unit of BW-100.
• Payment processed successfully (TX-XXXXXXXXXX).
• Inventory updated (BW-100 stock now 0).

# Optional Enhancements

- Add a refund task if payment succeeds but fulfillment fails later.

- Add concurrency-safe inventory locking if multiple agents act in parallel.

- Add a stateful Planner that uses Langroid context to check prior inventory before planning tasks.

# To-do 2
make your multi-agent e-commerce system realistic and stateful across sessions. Let’s go step by step, with actual notebook-ready code. We’ll replace the in-memory inventory dict with a SQLite database.

1️⃣ Setup SQLite DB
import sqlite3
import time

## Connect to a local SQLite DB (or create it if not exists)
conn = sqlite3.connect("inventory.db")
cursor = conn.cursor()

## Create inventory table
cursor.execute("""
CREATE TABLE IF NOT EXISTS inventory (
    sku TEXT PRIMARY KEY,
    name TEXT,
    stock INTEGER
)
""")
conn.commit()

## Optional: seed some products (only if not exists)
products = [
    ("BW-100", "Blue Widget", 1),
    ("AC-200", "Accessory 200", 5)
]

for sku, name, stock in products:
    cursor.execute("INSERT OR IGNORE INTO inventory (sku, name, stock) VALUES (?, ?, ?)", (sku, name, stock))
conn.commit()


✅ Now you have a persistent table inventory that survives across sessions.

2️⃣ Inventory Helper Functions

Replace your old in-memory functions with SQLite-backed functions:

def check_inventory_db(sku: str, qty: int) -> bool:
    cursor.execute("SELECT stock FROM inventory WHERE sku = ?", (sku,))
    row = cursor.fetchone()
    if row is None:
        return False
    return row[0] >= qty


def update_inventory_db(sku: str, qty: int) -> bool:
    cursor.execute("SELECT stock FROM inventory WHERE sku = ?", (sku,))
    row = cursor.fetchone()
    if row is None or row[0] < qty:
        return False
    new_stock = row[0] - qty
    cursor.execute("UPDATE inventory SET stock = ? WHERE sku = ?", (new_stock, sku))
    conn.commit()
    return True


✅ These ensure inventory updates persist across notebook restarts.

3️⃣ Modify CrewAgentWrapper / scheduler

Wherever you handled the "order" role, replace the in-memory logic with SQLite checks:

if role == "order":
    sku = params.get("sku")
    qty = int(params.get("quantity", 1))

    if not check_inventory_db(sku, qty):
        return {"status": "failed", "reason": "insufficient_stock", "available": 0}

    # Deduct stock in DB
    success = update_inventory_db(sku, qty)
    if not success:
        return {"status": "failed", "reason": "inventory_update_failed"}

    # Save order in memory / file
    order = {"order_id": f"ORD-{int(time.time())}", "sku": sku, "quantity": qty}
    mem_add("orders", order)
    path = file_writer_tool(f"order_{order['order_id']}.json", json.dumps(order))
    return {"status": "ok", "order": order, "file": path}

4️⃣ Skip payment if order fails

Same as before — check the last order result:

if role == "payment":
    last_order = next((r for r in memory.get("orders", [])[::-1]), None)
    if last_order is None:
        return {"status": "skipped", "reason": "No successful order found"}
    
    card = params.get("card_number", "")
    approved = card.endswith("42") or card.endswith("4242")
    tx = {"tx_id": f"TX-{int(time.time())}", "approved": approved}
    mem_add("events", {"type": "payment", "tx": tx})
    if approved:
        return {"status": "ok", "tx": tx}
    else:
        return {"status": "failed", "reason": "declined"}

5️⃣ Test multi-turn persistence
user_goal1 = "Place an order for 1 unit of BW-100 and charge card 4242424242424242."
multi_out1 = handle_user_goal_with_crew(user_goal1)

user_goal2 = "Place an order for 1 unit of BW-100 and charge card 4242424242424242."
multi_out2 = handle_user_goal_with_crew(user_goal2)

print(json.dumps(multi_out1, indent=2))
print(json.dumps(multi_out2, indent=2))


Expected behavior:

First run: order succeeds, payment succeeds, stock updated to 0.

Second run: order fails due to insufficient stock, payment skipped.

✅ Advantages

Persistence: Inventory survives notebook restarts.

Multi-turn awareness: Multiple user goals processed sequentially respect stock changes.

Realism: System now behaves like an actual e-commerce backend.